# Advent of code Day 9 - Rope Bridge

### rules 
* tail must add next to head every step that separates them
* special cases of note:
    * diagonal is considered touching, so a add by H which results in T not sharing the same row and column but only 1 away is fine
    * any separation of HT while not int he same row and col that is beyond 1 T adds diagonally to close hte gap
    * allow overlaps

## Part 1 - Number of unique points T has visited
- include start point
- play H adds one at a time
- adds are defined as 
    - left ( 0,-1)
    - right ( 0, 1)
    - up ( 1, 0)
    - down (-1, 0)

```
x   u2l u2  u2r x
ul2 .   .   .   ur2
l2  .   T   .   r2
dl2 .   .   .   dr2
x   d2l down   d2r x
```
- if H adds within the coords of T or its neighbors ( the dots or T), T does nothing
- if H adds the ring outside the neighbors T must act ( the letters )
- H cannot reach the xs if we move one spot at a time

- may be helpful to keep an absolute coord of T for recording (to a set), and a relative coord on H from T
    - start point can be arbitrarily set to start

In [212]:
def get_inputs(filename): 
    with open(filename) as file: 
        lines = file.read().strip()
    commands = [ left.split(' ') for left in lines.split('\n') ]
    return ''.join([ c[0] * int(c[1]) for c in commands ])
    
assert get_inputs('../data/Day09-pre.txt') == 'RRRRUUUULLLDRRRRDLLLLLRR'

In [213]:
# points as (row, col)
# u2l2 u2l u2 u2r u2r2
# ul2 .   .   .   ur2
# l2  .   T   .   r2
# dl2 .   .   .   dr2
# d2l2 d2l d  d2r d2r2

def add(x,y): return (x[0] + y[0], x[1] + y[1])
def subtract(x,y): return (x[0] - y[0], x[1] - y[1])

# relative coordinates
start,left,right,up,down   = (0,0),( 0,-1), ( 0, 1), ( 1, 0), (-1, 0)
ul,ur,dl,dr     = add(up,left), add(up,right), add(down,left), add(down,right)
u2,d2,r2,l2     = add(up,up), add(down,down), add(right,right), add(left,left)
u2l,u2r,ul2,ur2 = add(u2,left),add(u2,right),add(up,l2),add(up,r2)
d2l,d2r,dl2,dr2 = add(d2,left),add(d2,right),add(down,l2),add(down,r2)
u2l2,u2r2       = add(u2l,l), add(u2r,r)
d2l2,d2r2       = add(d2l,l), add(d2r,r)

def move(p, m):
    if   m == 'U': return add(p, up)
    elif m == 'D': return add(p, down)
    elif m == 'L': return add(p, left)
    elif m == 'R': return add(p, right)
    raise Exception(f'unknown move {m}')

def follow(mover,follower):
    diff = subtract(mover,follower)
    if   diff == u2:  return add(follower,up)
    elif diff == u2r: return add(follower,ur)
    elif diff == ur2: return add(follower,ur)
    elif diff == r2:  return add(follower,right)
    elif diff == dr2: return add(follower,dr)
    elif diff == d2r: return add(follower,dr)
    elif diff == d2:  return add(follower,down)
    elif diff == d2l: return add(follower,dl)
    elif diff == dl2: return add(follower,dl)
    elif diff == l2:  return add(follower,left)
    elif diff == ul2: return add(follower,ul)
    elif diff == u2l: return add(follower,ul)
    elif diff == u2l2: return add(follower,ul)
    elif diff == u2r2: return add(follower,ur)
    elif diff == d2l2: return add(follower,dl)
    elif diff == d2r2: return add(follower,dr)
    return follower

def part1(direction):
    knot1, knot2, history = start, start, {start}
    for direction in direction:
        knot1 = move(knot1,direction)
        followed = follow(knot1,knot2)
        if knot2 != followed: 
            history.add(followed)
            knot2 = followed
    return len(history)

def part2(direction):
    knots   = [start   for i in range(10)]
    history = {start}
    for direction in direction:
        knots[0] = move(knots[0],direction)
        for i in range(9):
            j = i + 1
            followed = follow(knots[i],knots[j])
            if j == 9 and knots[j] != followed: history.add(followed)
            knots[j] = followed
    return len(history)
            
################################################################
### test: movements of head
for m,result in [('U',up),('D',down),('L',left),('R',right)]:
    assert move(start,m) == result

### test: tail must move movements
assert react(u2, start,set()) == (u2,  up,  {up})
assert react(u2r,start,set()) == (u2r, ur, {ur})
assert react(ur2,start,set()) == (ur2, ur, {ur})
assert react(r2, start,set()) == (r2,  right,  {right})
assert react(dr2,start,set()) == (dr2, dr, {dr})
assert react(d2r,start,set()) == (d2r, dr, {dr})
assert react(d2, start,set()) == (d2,  down,  {down})
assert react(d2l,start,set()) == (d2l, dl, {dl})
assert react(dl2,start,set()) == (dl2, dl, {dl})
assert react(l2, start,set()) == (l2,  left,  {left})
assert react(ul2,start,set()) == (ul2, ul, {ul})
assert react(u2l,start,set()) == (u2l, ul, {ul})

### test: tail does not move
for h in [start, up, left, right, down, ur, ul, dr, dl]:
    assert react(h, start, set())  == (h, start, set())

### demo data
assert part1(get_inputs("../data/Day09-pre.txt")) == 13
assert part2(get_inputs("../data/Day09-pre.txt")) == 1

################################################################
### answers
print(f'Part 1: { part1(get_inputs("../data/Day09.txt")) }')
print(f'Part 2: { part2(get_inputs("../data/Day09.txt")) }')


# def print_knots(rows,cols,knots):
#     lines = ['.' * cols for row in range(rows)]
#     ki = len(knots)
#     for i in reversed(range(ki)):
#         r,c = knots[i]
#         l = lines[r]
#         lines[r] = f'{l[:c]}{i}{l[c+1:]}'
#     for l in reversed(lines):
#         print(l)

Part 1: 6384
Part 2: 2734
